In [ ]:
import gspread
from google.auth import default
from google.colab import auth

# Authenticate and authorize access to Google Sheets
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open the spreadsheet and select the worksheet
spreadsheet_id = '1fZwXtdgPClD43DjP4Rn2Th_bw76RU8HB6tV67oQcLxI'
worksheet_name = 'nikkei_headlines.csv'  # Replace with the actual sheet name
worksheet = gc.open_by_key(spreadsheet_id).worksheet(worksheet_name)

# Iterate through the results and update the spreadsheet
row_index = 1  # Start from row 1 (the first row)

In [ ]:
column_d_values = worksheet.col_values(4)  # 3 represents column C

In [ ]:
print(column_d_values)

['Neutral', 'Neutral', 'Neutral', 'Positive', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Positive', 'Neutral', 'Neutral', 'Neutral', 'Negative', 'Neutral', 'Neutral', 'Negative', 'Neutral', 'Neutral', 'Negative', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Positive', 'Neutral', 'Neutral', 'Negative', 'Negative', 'Negative', 'Neutral', 'Neutral', 'Negative', 'Positive', 'Neutral', 'Positive', 'Neutral', 'Positive', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Negative', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Positive', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Negative', 'Neutral', 'Positive', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Negative', 'Positive', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Negative', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Negative',

In [ ]:
from collections import Counter
counts = Counter(column_d_values)
print(counts)

Counter({'': 26731, 'Neutral': 1902, 'Negative': 425, 'Positive': 265})


In [2]:
!pip install yfinance

In [27]:
import yfinance as yf

# Define the ticker symbol for Nikkei 225
ticker = "^N225"

# # Set the start and end dates
# start_date = "2000-01-01"
# end_date = "2024-01-01"

# # Download the data
# data = yf.download(ticker, start=start_date, end=end_date)

# # Display the downloaded data
# print(data)


In [28]:
# data.to_csv('nikkei_headlines.csv')

In [5]:
import gspread
from google.auth import default
from google.colab import auth

# Authenticate and authorize access to Google Sheets
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open the spreadsheet and select the worksheet
spreadsheet_id = '1fZwXtdgPClD43DjP4Rn2Th_bw76RU8HB6tV67oQcLxI'
worksheet_name = 'nikkei_headlines.csv'  # Replace with the actual sheet name
worksheet = gc.open_by_key(spreadsheet_id).worksheet(worksheet_name)

# Iterate through the results and update the spreadsheet
row_index = 1  # Start from row 1 (the first row)

In [ ]:
from datetime import datetime, timedelta
from tqdm import tqdm

for row_index, row in enumerate(tqdm(worksheet.get_all_values()), start=1):
  if row_index <= 2878:
  # if row_index <= 28670: # End part
    continue
  # if row_index >= 2325:
  #   break
  row_values = worksheet.row_values(row_index)  # access to gsheet is 1-idx
  # print(row_values[3])
  # print(row_values[2])
  try:
    label = row_values[3]
  except IndexError:
    # print(f"Skipping row {row_index} due to missing data.")
    continue  # Skip to the next iteration

  if row_values[3] == 'Neutral':
    continue
  date = row_values[1]

  try:
    Movement = row_values[9]
    continue
  except IndexError:
    pass

  date_object = datetime.strptime(date, '%Y-%m-%d')
  next_day = date_object + timedelta(days=1)
  if date_object.weekday() >= 5:  # 5 and 6 correspond to Saturday and Sunday
    worksheet.update_cell(row_index, 9, "trading holiday")
    continue
  # Handle weekends
  while next_day.weekday() >= 5:  # 5 and 6 correspond to Saturday and Sunday
    next_day += timedelta(days=1)
  next_day_str = next_day.strftime('%Y-%m-%d')
  next_next_day_str = next_day + timedelta(days=1)
  next_next_day_str = next_next_day_str.strftime('%Y-%m-%d')
  print(f"next_day_str: {next_day_str}")
  print(f"next_next_day_str: {next_next_day_str}")

  data = yf.download(ticker, start=date, end=next_next_day_str);
  # data = yf.download(ticker, start=date, end=next_day_str);

  print(data)

  close_values = data['Close']
  # print(close_values)

  # Get the Close values for the first and second days
  # Skip if the news item publication day is a trading holiday
  try:
    first_day_close = close_values[date_object]
  except KeyError:
    print(f"Skipping row {row_index} due to missing data for {date_object}")
    worksheet.update_cell(row_index, 9, "trading holiday")
    continue

  # If the day after publication is a trading holiday, use the next trading day instead
  temp_day = next_day
  second_day_close = None
  while second_day_close is None:
    try:
      temp_day_str = temp_day.strftime('%Y-%m-%d')
      next_temp_day_str = temp_day + timedelta(days=1)
      next_temp_day_str = next_temp_day_str.strftime('%Y-%m-%d')
      data = yf.download(ticker, start=temp_day_str, end=next_temp_day_str)
      close_values = data['Close']
      second_day_close = close_values[temp_day_str]
    except KeyError:
      print(f"Data for {temp_day_str} is missing. Trying next day.")
      temp_day += timedelta(days=1)

  # first_day_close = close_values[date_object]
  # second_day_close = close_values[next_day_str]
  # first_day_close = close_values.iloc[0]
  # second_day_close = close_values.iloc[1]


  # Calculate the difference
  difference = second_day_close - first_day_close

  if difference > 0:
    result = "Up"
  elif difference < 0:
    result = "Down"
  else:
    result = "Constant"

  # print(f"Market movement from first to second day: {result}")

  print("Writing...")
  worksheet.update_cell(row_index, 7, first_day_close)
  worksheet.update_cell(row_index, 8, second_day_close)
  worksheet.update_cell(row_index, 9, result)